# Tutorial: Quantum Embedding Theory (QDET) Calculations with WEST

In this tutorial, we will calculate the correlated excitations of the $\mathrm{NV^-}$ defect in Diamond using Quantum Defect Embedding Theory (QDET). In QDET, we formulate a many-body Hamiltonian $\mathrm{H}^{eff}$ for a selected set of defect orbitals of the form
$$
\mathrm{H}^{eff} = \sum_{ij}^A t^{eff}_{ij}\hat{a}_i^\dagger \hat{a}_j + \frac{1}{2} \sum_{ijkl}^A v^{eff}_{ijkl} \hat{a}_i^\dagger \hat{a}_j^\dagger \hat{a}_l \hat{a}_k,
$$
where $t^{eff}$ and $v^{eff}$ are the effective one- and two-body terms, respectively. The solution of the effective Hamiltonian then yields the excitations of the spin defect. More details can be found in <a href="https://doi.org/10.1021/acs.jctc.2c00240">Nan Sheng, Christian Vorwerk, Marco Govoni, and Giulia Galli, J. Chem. Theory Comput. 18, 6, 3512 (2022)</a>.

QDET calculations are performed in two separate steps:
1. In an initial WEST calculation, the parameters $t^{eff}$ and $v^{eff}$ are calculated and stored in a JSON file.
2. The effective Hamiltonian is constructed from the parameters in the JSON file and is solved by exact diagonalization. This step is performed by the WESTpy library.

## Step 1: Parameters of the Effective Hamiltonian

### Step 1.1:  Mean-field Starting Point

As a first step, we have to perform the mean-field electronic structure calculations within density-functional theory (DFT) using the <a href='https://www.quantum-espresso.org/'>Quantum ESPRESSO</a> code.

Download the following files in your working directory:

In [1]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.0.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.0.upf

We can now inspect the `pw.in` file, the input for the `pw.x` code:

In [2]:
%%bash
cat pw.in

&CONTROL
calculation = 'scf'
wf_collect = .true.
pseudo_dir = './'
/
&SYSTEM
input_dft = 'PBE'
ibrav = 0
ecutwfc = 50
nosym = .true.
tot_charge = -1
nspin = 1
nbnd = 176
occupations = 'from_input'
nat = 63
ntyp = 2
/
&ELECTRONS
conv_thr = 1D-08
/
K_POINTS gamma
CELL_PARAMETERS angstrom
7.136012  0.000000  0.000000
0.000000  7.136012  0.000000
0.000000  0.000000  7.136012
ATOMIC_SPECIES
C  12.01099968  C_ONCV_PBE-1.0.upf
N  14.00699997  N_ONCV_PBE-1.0.upf
ATOMIC_POSITIONS crystal
C    0.99996000  0.99996000  0.99996000
C    0.12495000  0.12495000  0.12495000
C    0.99905000  0.25039000  0.25039000
C    0.12350000  0.37499000  0.37499000
C    0.25039000  0.99905000  0.25039000
C    0.37499000  0.12350000  0.37499000
C    0.25039000  0.25039000  0.99905000
C    0.37499000  0.37499000  0.12350000
C    0.00146000  0.00146000  0.50100000
C    0.12510000  0.12510000  0.62503000
C    0.00102000  0.24944000  0.74960000
C    0.12614000  0.37542000  0.87402000
C    0.24944000  0.00102000  0.74960

We can now run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

### Step 1.2: Calculation of Dielectric Screening

As for GW calculations with WEST, we first have to determine the dielectric screening before we can proceed to calculate the excitations of the spin defect. In WEST, the dielectric screening is obtained from the projective dielectric eigendecomposition (PDEP) technique. The calculation with `wstat.x` requires an input file `wstat.in`.

You can download the input file with the following command:

In [3]:
%%bash 
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wstat.in

Once again, we can have a look at the input file:

In [4]:
%%bash
cat wstat.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001


As we can see, there are no input parameters in `wstat.in` specific to QDET. We can now execute `wstat.x` with the following command.

In [ ]:
%%bash
mpirun -n 2 wstat.x -i wstat.in > wstat.out

### Step 1.3: QDET Calculation

The actual QDET calculation in WEST is performed with the `wfreq.x` command. The calculation includes the calculation of the $\mathrm{G_0W_0}$ quasiparticle energies, that of the partially screened Coulomb potential, and finally the construction of an effective Hamiltonian.

In [5]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.in

Let us again inspect the file `wfreq.in`:

In [6]:
%%bash
cat wfreq.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001

wfreq_control:
  wfreq_calculation: XWGQH
  macropol_calculation: C
  l_enable_off_diagonal: true
  n_pdep_eigen_to_use: 512
  qp_bands: [87, 122, 123, 126, 127, 128]
  n_refreq: 300
  ecut_refreq: 2.0


The QDET calculation is triggered by the keyword `H` (for effective Hamiltonian) in the element `wfreq_calculation`. The DFT states that define the active space are defined with the element `qp_bands` (in our case states 87, 122, 123, 126, 127, and 128). The active space is chosen to be a set of localized defect orbitals in the supercell. For the $\mathrm{NV}^-$ center in the 63-atom supercell, we have chosen the orbitals with the largest localization factor. See the tutorial on the calculation of the localization factor <a href="http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/basic_006.ipynb">here</a>. 

For QDET, it is important to set the flag `l_enable_off_diagonal = True` which allows for the calculation of off-diagonal entries of the self-energy.

The `wfreq.x` calculation can be performed as

In [ ]:
%%bash
mpirun -n 2 wfreq.x -i wfreq.in > wfreq.out

If the reader does **NOT** have the computational ressource to run the calculation, the WEST JSON file can be downloaded as

In [7]:
%%bash
mkdir west.wfreq.save
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.json -O west.wfreq.save/wfreq.json

## Step 2: Exact Diagonalization of the Effective Hamiltonian

The effective Hamiltonian is constructed and diagonalized using the WESTpy code.

In [10]:
import json
import numpy as np

from westpy.qdet import QDETResult

# construct object for effective Hamiltonian
effective_hamiltonian = QDETResult(filename='west.wfreq.save/wfreq.json')

# diagonalize Hamiltonian
solution = effective_hamiltonian.solve()

-----------------------------------------------------
Building effective Hamiltonian...
nspin: 1
occupations: [[2. 2. 2. 2. 1. 1.]]


diag[1RDM - 1RDM(GS)]                              
  E [eV] char                    87   122   123   126   127   128
0  0.000   3-                   0.0  -0.0  -0.0  -0.0  -1.0  -1.0
1  0.436   1-                   0.0  -0.0  -0.0  -0.1  -1.0  -0.9
2  0.436   1-                   0.0  -0.0  -0.0  -0.1  -0.9  -1.0
3  1.250   1-                   0.0  -0.0  -0.0  -0.1  -0.9  -0.9
4  1.941   3-                   0.0  -0.0  -0.1  -0.9  -1.0   0.0
5  1.941   3-                   0.0  -0.0  -0.1  -0.9   0.0  -1.0
6  2.937   1-                   0.0  -0.0  -0.0  -0.9  -0.9  -0.2
7  2.937   1-                   0.0  -0.0  -0.0  -0.9  -0.2  -0.9
8  4.662   1-                   0.0  -0.0  -0.2  -1.7  -0.0  -0.0
9  5.073   3-                   0.0  -0.7  -0.2  -0.1  -1.0   0.0

-----------------------------------------------------


Calling the function `solve()` of the object `QDETResult` writes the excitation energies, spin multiplicity and relative occupation (compared to the occupation of the groundstate) to screen.

Let us consider the first excited state as an example: It has an energy of 0.436 eV above the groundstate, has spin multiplicity 1 (i.e. it is a singlet), and is two-fold degenerate. It has nearly the same occupation as the groundstate, only the occupation in band 126 is reduced by 0.1, while that of 128 is increased by 0.1. Since the state has the same occupation as the groundstate, it is a *multiplet* excitation.

Much more detailed about the excitations is stored in the dictionary `solution`:

In [11]:
print(solution.keys())

dict_keys(['nstates', 'evs_au', 'evs', 'evcs', 'mults', 'rdm1s'])


Let us consider its keys:
- `nstates` yields the number of calculated excitations.
- `evs` stores the excitation energies relative to the groundstate in eV.
- `evs_au` stores the absolute energies of the many-body states in Hartree.
- `mults` stores the spin multiplicity for each state.
- `rdm1s` stores the spin-summed reduced 1-body density matrix for each many-body state.
- `evcs` stores the many-body states in 2nd quantization.